### Detecting Memes

Adopted from the paper: _MEMETECTOR: ENFORCING DEEP FOCUS FOR MEME DETECTION_

_url_: https://arxiv.org/abs/2205.13268



In [79]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

img_size = 250
input_shape = (img_size, img_size, 3)
patch_size = 25
num_patches = (img_size // patch_size) ** 2
projection_dim = 64
batch_size = 64


class Patches(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def get_config(self):
        config = super().get_config().copy()
        config.update({"patch_size": self.patch_size})
        return config

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


class PatchEncoder(layers.Layer):
    def __init__(self, projection_dim, **kwargs):
        super(PatchEncoder, self).__init__()
        self.projection_dim = projection_dim
        self.projection = layers.Dense(units=projection_dim)
        self.class_token = tf.Variable(
            initial_value=tf.zeros_initializer()(
                shape=(1, 1, projection_dim), dtype="float32"
            ),
            trainable=True,
        )

    def get_config(self):
        config = super().get_config().copy()
        config.update({"projection_dim": self.projection_dim})
        return config

    def call(self, patch):
        batch_size = tf.shape(patch)[0]
        cls_broadcasted = tf.cast(
            tf.broadcast_to(self.class_token, [batch_size, 1, self.projection_dim]),
            dtype=patch.dtype,
        )
        return tf.concat([cls_broadcasted, self.projection(patch)], 1)


class PositionalEmbedding(layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super(PositionalEmbedding, self).__init__()
        self.projection_dim = projection_dim
        self.num_patches = num_patches
        self.position_embedding = layers.Embedding(
            input_dim=num_patches + 1, output_dim=projection_dim
        )

    def get_config(self):
        config = super().get_config().copy()
        config.update(
            {"projection_dim": self.projection_dim, "num_patches": self.num_patches}
        )
        return config

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches + 1, delta=1)
        batch_size = tf.shape(patch)[0]
        embs = tf.cast(
            tf.broadcast_to(
                self.position_embedding(positions),
                [batch_size, self.num_patches + 1, self.projection_dim],
            ),
            dtype=patch.dtype,
        )
        return patch + embs



In [80]:
# load the model
model = load_model('ConcepCap 0.67 TXT 1.00 ViTa.h5',
            custom_objects={"Patches": Patches,
            "PatchEncoder": PatchEncoder,
            "PositionalEmbedding": PositionalEmbedding,
            "optimizer": "tfa.optimizers.AdamW"},
)

In [81]:
print(model.summary())

Model: "model_47"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_48 (InputLayer)           [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
patches_10 (Patches)            (None, None, 1875)   0           input_48[0][0]                   
__________________________________________________________________________________________________
patch_encoder_10 (PatchEncoder) (None, None, 64)     120128      patches_10[0][0]                 
__________________________________________________________________________________________________
positional_embedding_10 (Positi (None, 101, 64)      6464        patch_encoder_10[0][0]           
___________________________________________________________________________________________

In [88]:
# I am not sure about this part yet. Working on it.
import cv2
import numpy as np

# dimensions of our images.
img_width, img_height = 250, 250

img = cv2.imread('img0.jpg')
img = cv2.resize(img, (img_width, img_height))
img = np.reshape(img, [1, img_height, img_width, 3])

prediction = model.predict(img)

print(prediction)




[[0.]]
